In [1]:
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import torchvision
import math
from torch.autograd import Variable
import copy
import torchvision.transforms as transforms

In [3]:
import pandas as pd 
import numpy as np

In [ ]:
#not using bours of restaurants, since we have no information on what time people are looking to eat

In [131]:
rating_dir = './RCdata/rating_final.csv'
rest_profile_dir = './RCdata/geoplaces2.csv'
rest_cuisine_dir = './RCdata/chefmozcuisine.csv'
rest_payment_dir = './RCdata/chefmozaccepts.csv'
rest_parking_dir = './RCdata/chefmozparking.csv'
user_cuisine_dir = './RCdata/usercuisine.csv'
user_payment_dir = './RCdata/userpayment.csv'
user_profile_dir = './RCdata/userprofile.csv'

In [132]:
rest_profile = pd.read_csv(rest_profile_dir, encoding = 'cp1252')
rest_cuisine = pd.read_csv(rest_cuisine_dir) #encoding = 'cp1252')
rest_payment = pd.read_csv(rest_payment_dir)
rest_parking = pd.read_csv(rest_parking_dir)

In [133]:
rest_parking.columns

Index(['placeID', 'parking_lot'], dtype='object')

In [134]:
n_rests = len(rest_profile.placeID)
print (n_rests)

130


In [135]:
rests = np.array(rest_profile.placeID)

In [136]:
cuisine_data = [0 for i in range(n_rests)]

In [137]:
for i in range(len(cuisine_data)):
    cuisines = rest_cuisine[rest_cuisine['placeID'] == rests[i]]['Rcuisine']
    this_one = ''
    if len(cuisines)>0: 
        for j in range(len(cuisines)):
            this_one = this_one + cuisines.iloc[j] + ' '        
    cuisine_data[i] = this_one

In [138]:
rest_profile.loc[:,'Cuisine'] = pd.Series(cuisine_data, index=rest_profile.index)

In [139]:
payment_data = [0 for i in range(n_rests)]

In [140]:
for i in range(len(payment_data)):
    payment = rest_payment[rest_payment['placeID'] == rests[i]]['Rpayment']
    this_one = ''
    if len(payment)>0: 
        for j in range(len(payment)):
            this_one = this_one + payment.iloc[j] + ' '        
    payment_data[i] = this_one

In [141]:
rest_profile.loc[:,'Payment'] = pd.Series(payment_data, index=rest_profile.index)

In [142]:
parking_data = [0 for i in range(n_rests)]

In [143]:
for i in range(len(parking_data)):
    parking = rest_parking[rest_parking['placeID'] == rests[i]]['parking_lot']
    this_one = ''
    if len(parking)>0: 
        for j in range(len(parking)):
            this_one = this_one + parking.iloc[j] + ' '        
    parking_data[i] = this_one

In [144]:
rest_profile.loc[:,'Parking'] = pd.Series(parking_data, index=rest_profile.index)

In [145]:
rest_profile.shape

(130, 24)

In [146]:
rest_profile.columns

Index(['placeID', 'latitude', 'longitude', 'the_geom_meter', 'name', 'address',
       'city', 'state', 'country', 'fax', 'zip', 'alcohol', 'smoking_area',
       'dress_code', 'accessibility', 'price', 'url', 'Rambience', 'franchise',
       'area', 'other_services', 'Cuisine', 'Payment', 'Parking'],
      dtype='object')

In [147]:
user_profile = pd.read_csv(user_profile_dir)
user_payment = pd.read_csv(user_payment_dir)
user_cuisine = pd.read_csv(user_cuisine_dir)

In [148]:
user_cuisine.columns

Index(['userID', 'Rcuisine'], dtype='object')

In [149]:
n_users = len(user_profile.userID)
print (n_users)

138


In [150]:
users = np.array(user_profile.userID)

In [151]:
cuisine_data = [0 for i in range(n_users)]

In [152]:
for i in range(len(cuisine_data)):
    cuisines = user_cuisine[user_cuisine['userID'] == users[i]]['Rcuisine']
    this_one = ''
    if len(cuisines)>0: 
        for j in range(len(cuisines)):
            this_one = this_one + cuisines.iloc[j] + ' '        
    cuisine_data[i] = this_one

In [153]:
user_profile.loc[:,'Cuisine'] = pd.Series(cuisine_data, index=user_profile.index)

In [154]:
payment_data = [0 for i in range(n_users)]

In [155]:
for i in range(len(payment_data)):
    payment = user_payment[user_payment['userID'] == users[i]]['Upayment']
    this_one = ''
    if len(payment)>0: 
        for j in range(len(payment)):
            this_one = this_one + payment.iloc[j] + ' '        
    payment_data[i] = this_one

In [156]:
user_profile.loc[:,'Payment'] = pd.Series(payment_data, index=user_profile.index)

In [157]:
user_profile

,userID,latitude,longitude,smoker,drink_level,dress_preference,ambience,transport,marital_status,hijos,...,interest,personality,religion,activity,color,weight,budget,height,Cuisine,Payment
0,U1001,22.139997,-100.978803,false,abstemious,informal,family,on foot,single,independent,...,variety,thrifty-protector,none,student,black,69,medium,1.77,American,cash
1,U1002,22.150087,-100.983325,false,abstemious,informal,family,public,single,independent,...,technology,hunter-ostentatious,Catholic,student,red,40,low,1.87,Mexican,cash
2,U1003,22.119847,-100.946527,false,social drinker,formal,family,public,single,independent,...,none,hard-worker,Catholic,student,blue,60,low,1.69,Mexican,cash
3,U1004,18.867000,-99.183000,false,abstemious,informal,family,public,single,independent,...,variety,hard-worker,none,professional,green,44,medium,1.53,Bakery Breakfast-Brunch Japanese Contemporary ...,cash bank_debit_cards
4,U1005,22.183477,-100.959891,false,abstemious,no preference,family,public,single,independent,...,none,thrifty-protector,Catholic,student,black,65,medium,1.69,American,cash
5,U1006,22.150000,-100.983000,true,social drinker,no preference,friends,car owner,single,independent,...,variety,hard-worker,none,student,blue,75,medium,1.80,Mexican,cash
6,U1007,22.118464,-100.938256,false,casual drinker,informal,solitary,public,single,independent,...,variety,thrifty-protector,Catholic,student,purple,60,low,1.59,Family,cash
7,U1008,22.122989,-100.923811,false,social drinker,formal,solitary,public,single,independent,...,technology,hard-worker,Catholic,student,green,68,low,1.72,Cafe-Coffee_Shop Mexican Juice Hawaiian Hot_Do...,cash
8,U1009,22.159427,-100.990448,false,abstemious,formal,family,on foot,single,kids,...,variety,thrifty-protector,Catholic,student,green,75,medium,1.78,Diner Fast_Food Family Cafe-Coffee_Shop Deli-S...,cash
9,U1010,22.190889,-100.998669,false,social drinker,no preference,friends,car owner,married,kids,...,technology,hard-worker,none,student,green,40,medium,1.67,Mexican,cash


In [158]:
user_profile.columns

Index(['userID', 'latitude', 'longitude', 'smoker', 'drink_level',
       'dress_preference', 'ambience', 'transport', 'marital_status', 'hijos',
       'birth_year', 'interest', 'personality', 'religion', 'activity',
       'color', 'weight', 'budget', 'height', 'Cuisine', 'Payment'],
      dtype='object')

In [159]:
rest_profile.columns

Index(['placeID', 'latitude', 'longitude', 'the_geom_meter', 'name', 'address',
       'city', 'state', 'country', 'fax', 'zip', 'alcohol', 'smoking_area',
       'dress_code', 'accessibility', 'price', 'url', 'Rambience', 'franchise',
       'area', 'other_services', 'Cuisine', 'Payment', 'Parking'],
      dtype='object')

In [160]:
ratings = pd.read_csv(rating_dir)

In [161]:
from sklearn.model_selection import train_test_split
rating_train, rating_test = train_test_split(ratings, test_size=0.3)    

In [162]:
n_users = len(ratings.userID.unique())
n_items = len(ratings.placeID.unique())
print (n_users)
print (n_items)

138
130


In [163]:
user_list = np.array(ratings.userID.unique())
item_list = np.array(ratings.placeID.unique())

In [210]:
user_profile.columns

Index(['userID', 'latitude', 'longitude', 'smoker', 'drink_level',
       'dress_preference', 'ambience', 'transport', 'marital_status', 'hijos',
       'birth_year', 'interest', 'personality', 'religion', 'activity',
       'color', 'weight', 'budget', 'height', 'Cuisine', 'Payment'],
      dtype='object')

In [180]:
rest_profile.columns

Index(['placeID', 'latitude', 'longitude', 'the_geom_meter', 'name', 'address',
       'city', 'state', 'country', 'fax', 'zip', 'alcohol', 'smoking_area',
       'dress_code', 'accessibility', 'price', 'url', 'Rambience', 'franchise',
       'area', 'other_services', 'Cuisine', 'Payment', 'Parking'],
      dtype='object')

In [257]:
def get_context(user, rest, user_profile, rest_profile):
    this_user = user_profile[user_profile['userID']==user]
    this_rest = rest_profile[rest_profile['placeID'] == rest]
    
    #print (this_user)
    #print (this_rest)
    contexts = 0
    
    if this_user['smoker'].item() == 'true' and (this_rest['smoking_area'].item() == 'section'or this_rest['smoking_area'].item() == 'permitted' or this_rest['smoking_area'].item() == 'only at bar'):
        contexts = contexts + 1
    
    if (this_user['drink_level'].item() == 'casual drinker' or this_user['drink_level'].item() == 'social_drinker') and this_rest['alcohol'].item() == 'Wine-Beer' or this_rest['alcohol'].item() == 'Full_Bar':
        contexts = contexts + 1 
    
    if (this_user['dress_preference'].item() == 'formal' or this_user['dress_preference'].item() == 'elegant') and (this_rest['dress_code'].item() == 'formal' or this_rest['Rambience'].item() == 'quiet' or this_rest['franchise'].item()== 'f'):
        contexts = contexts + 1 
        
    if (this_user['dress_preference'].item() == 'informal') and (this_rest['dress_code'].item() == 'informal' or this_rest['dress_code'].item() == 'casual'):
        contexts = contexts + 1 
    
    if (this_user['ambience'].item() == 'family' or this_user['ambience'].item() == 'friends') and (this_rest['Rambience'].item() == 'familiar'):
        contexts = contexts + 1 
    
    if (this_user['ambience'].item() == 'solitary') and (this_rest['Rambience'].item() == 'quiet'):
        contexts = contexts + 1
    
    if (this_user['transport'].item() == 'car owner') and (this_rest['Parking'].item() == 'yes' or this_rest['Parking'].item() == 'public' or this_rest['Parking'].item() == 'valet parking'):
        contexts = contexts + 1 
        
    if (this_user['activity'].item() == 'student' or this_user['activity'].item() == 'working class') and (this_rest['price'].item() == 'low' or this_rest['price'].item() == 'medium'):
        contexts = contexts + 1 
    
    if (this_user['activity'].item() == 'unemployed') and (this_rest['price'].item() == 'low'):
        contexts = contexts + 1 
    
    if (this_user['activity'].item() == 'professional') and (this_rest['price'].item() == 'medium' or this_rest['price'].item() == 'high'):
        contexts = contexts + 1 
    
    if (this_user['budget'].item() == 'high') and (this_rest['price'].item() == 'high'):
        contexts = contexts + 1 
    
    if (this_user['budget'].item() == 'medium') and (this_rest['price'].item() == 'medium'):
        contexts = contexts + 1 
    
    if (this_user['budget'].item() == 'low') and (this_rest['price'].item() == 'low' or this_rest['franchise'].item() == 't'):
        contexts + contexts + 1 
        
    this_user_cuisines = this_user['Cuisine'].item().split(' ')
    this_rest_cuisines = this_rest['Cuisine'].item()
    
    this_user_payment = this_user['Payment'].item().split(' ')
    this_rest_payment = this_rest['Payment'].item()
    
    for j in range(len(this_user_cuisines)-1):
        if this_user_cuisines[j] in this_rest_cuisines:
            contexts = contexts + 1
    
    for m in range(len(this_user_payment)-1):
        if this_user_payment[m] in this_rest_payment:
            contexts = contexts + 1 
        
    return contexts
      

In [164]:
from scipy.sparse import rand as sprand
from scipy.sparse import lil_matrix

In [ ]:
#do we need to create contexts for the entirety of the restaurants? 

In [287]:
contexts_full = lil_matrix((n_users, n_items), dtype = float)
for i in range(len(user_list)):
    for j in range(len(item_list)):
        contexts_full[i, j] = get_context(user_list[i], item_list[j], user_profile, rest_profile)

In [264]:
interactions_full_orig = lil_matrix((n_users, n_items), dtype = float)
for row in ratings.itertuples():
    user, = np.where(user_list == row[1])
    item, = np.where(item_list == row[2])
    interactions_full_orig[user[0], item[0]] = row[3] 

In [258]:
interactions_train_orig = lil_matrix((n_users, n_items), dtype = float)
context_train = lil_matrix((n_users, n_items), dtype = float)
for row in rating_train.itertuples():
    user, = np.where(user_list == row[1])
    item, = np.where(item_list == row[2])
    context_train[user[0], item[0]] = get_context(row[1], row[2], user_profile, rest_profile)
    interactions_train_orig[user[0], item[0]] = row[3]

In [263]:
#original ratings
interactions_test_orig = lil_matrix((n_users, n_items), dtype = float)
context_test = lil_matrix((n_users, n_items), dtype = float)
for row in rating_test.itertuples():
    user, = np.where(user_list == row[1])
    item, = np.where(item_list == row[2])
    interactions_test_orig[user[0], item[0]] = row[3]
    context_test[user[0], item[0]] = get_context(row[1], row[2], user_profile, rest_profile)

In [349]:
class MatrixFactorization(torch.nn.Module):
    
    def __init__(self, n_users, n_items, n_factors=5):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users,
                                               n_factors,
                                               sparse=False)
        self.item_factors = torch.nn.Embedding(n_items,n_factors,sparse=False)
                  

    # I NEED TO IMPLEMENT THE PREDICTION based on context
    def predict(self, user, item):
        # Need to fit bias factors
        #pred = (self.user_factors(user) * self.item_factors(item)).sum(1)
        
        #context = (self.user_context(user) * self.item_(item)).sum(1)
        return (self.user_factors(user) * self.item_factors(item)).sum(1)
    
    # Much more efficient batch operator. This should be used for training purposes
    
    def forward(self, users, items, context):
        #return (self.user_factors(user) * self.item_factors(item)).sum(1)
    
        orig = torch.mm(self.user_factors(users),torch.transpose(self.item_factors(items),0,1))
        
        #here = torch.from_numpy(context)
        #context = torch.mm(self.user_context(users), torch.transpose(self.item_context(items),0,1))
        
        #fused_context = context * self.fusion.expand_as(context)
        
        mat = torch.add(orig, context)
        return mat
        #return torch.mm(self.user_factors(users),torch.transpose(self.item_factors(items),0,1))
    

In [348]:
class BiasedMatrixFactorization(torch.nn.Module):
    
    def __init__(self, n_users, n_items,mean, n_factors=5):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users,
                                               n_factors,
                                               sparse=False)
        self.item_factors = torch.nn.Embedding(n_items, n_factors,sparse=False)
        self.user_biases = torch.nn.Embedding(n_users, 1, sparse = False)
        
        self.item_biases = torch.nn.Embedding(n_items,1, sparse = False)
                                               
        self.mu = mean
    
    #need to change this predict function    
    def predict(self, user, item):
        pred = self.mu + self.user_biases(user) + self.item_biases(item)
        orig = (self.user_factors(user) * self.item_factors(item)).sum(1)
        final = pred + orig
        return final
    
    def forward(self, users, items, context):
        
        item_means = []
        user_means = []
        for i in range(len(users)):
            item_means.append(torch.t(self.item_biases(items)))
            
        for j in range(len(items)):
            user_means.append(torch.t(self.user_biases(users)))
        
        it_means = torch.cat(item_means, 0)
        us_means = torch.cat(user_means, 0)
        us_means = torch.t(us_means)
        
        orig = torch.mm(self.user_factors(users),torch.transpose(self.item_factors(items),0,1))
        
        y = torch.add(orig, it_means)
        
        f = torch.add(y,us_means)
        
        result = torch.add(f,self.mu)
        
        mat = torch.add(result, context)
        
        return mat

In [266]:
def get_batch(batch_size,ratings):
    # Sort our data and scramble it
    rows, cols = ratings.shape
    p = np.random.permutation(rows)
    
    # create batches
    sindex = 0
    eindex = batch_size
    while eindex < rows:
        batch = p[sindex:eindex]
        temp = eindex
        eindex = eindex + batch_size
        sindex = temp
        yield batch

    if eindex >= rows:
        batch = range(sindex,rows)
        yield batch

In [267]:
def load_model(model,load_path):
    load_dict = torch.load(load_path)
    val_loss = load_dict['val_loss']
    model.load_state_dict(load_dict['model_state_dict'])


def checkpoint_model(val_loss, model,save_path):
    save_dict = dict(
                     val_loss=val_loss,
                     model_state_dict=model.state_dict())
                     #opt_state_dict=predictions.state_dict())
    torch.save(save_dict, save_path)

In [335]:
def run_validation(model, test_ratings, context_mat, fusion, BATCH_SIZE, load_path):
    load_model(model, load_path)
    running_loss = 0.0
    loss_func = torch.nn.MSELoss()
    accur = 0
    for i,batch in enumerate(get_batch(BATCH_SIZE, test_ratings)):
        
        interactions = Variable(torch.FloatTensor(test_ratings[batch, :].toarray()))
        rows = Variable(torch.LongTensor(batch))
        cols = Variable(torch.LongTensor(np.arange(test_ratings.shape[1])))
        
        first = (context_mat[batch,:].toarray())*fusion
        contexts = Variable(torch.FloatTensor(first))
        
        
        predictions = model(rows, cols, contexts)
        loss = loss_func(predictions, interactions)
        
        running_loss += np.sqrt(loss.data[0])*BATCH_SIZE
        
        correct = get_accuracy_test(test_ratings, model, rows, fusion, context_mat)
        accur = accur+ correct
    
    epoch_loss = running_loss/test_ratings.shape[0]
    return epoch_loss,accur

In [269]:
#when using the food + service ratings (not used right now)
def run_validation_test(model, test_ratings, test_ratings_orig, BATCH_SIZE, load_path):
    load_model(model, load_path)
    running_loss = 0.0
    loss_func = torch.nn.MSELoss()
    accur = 0
    for i,batch in enumerate(get_batch(BATCH_SIZE, test_ratings)):
        
        interactions = Variable(torch.FloatTensor(test_ratings[batch, :].toarray()))
        rows = Variable(torch.LongTensor(batch))
        cols = Variable(torch.LongTensor(np.arange(test_ratings.shape[1])))
        
        predictions = model(rows, cols)
        loss = loss_func(predictions, interactions)
        
        running_loss += np.sqrt(loss.data[0])*BATCH_SIZE
        
        correct = get_accuracy_test(test_ratings_orig, model, rows)
        accur = accur+ correct
    
    epoch_loss = running_loss/test_ratings.shape[0]
    return epoch_loss,accur

In [336]:
def get_accuracy_test(interactions_orig,model,rows,fusion, contexts_full):
    
    checks = np.nonzero(interactions_orig)
    check_these = checks[0]
    
    batch = range(interactions_orig.shape[0])
    interactions = Variable(torch.FloatTensor(interactions_orig[batch,:].toarray()))
    users = Variable(torch.LongTensor(batch))
    cols = Variable(torch.LongTensor(np.arange(interactions_orig.shape[1])))
    
    first = (contexts_full[batch,:].toarray())*fusion
    contexts = Variable(torch.FloatTensor(first))
    
    predictions = model(users, cols, contexts)
    
    x = predictions.data.numpy()
    
    correct = 0
    
    for u in range(len(rows)):
        user = rows.data[u]
        where = np.where(check_these==user)
        index = where[0]
        if len(index) > 0:
            for c in range(len(index)):
                rest = checks[1][index[c]]
                prediction = predictions[u][rest]
                   
                val = prediction.data[0]
                
                if val <= np.percentile(x,22):
                    this_val = 0.0
                elif np.percentile(x,22) < val < np.percentile(x, 58):
                    this_val = 1.0
                elif val >= np.percentile(x, 58):
                     this_val= 2.0
        
                
                if this_val == interactions_orig[user,rest]:
                    correct = correct + 1 
            else:
                next
    return correct

In [310]:
batch_size = 10
save_path = './project_models/model.ckpt'
save_path_val = './project_models/modelv.ckpt'
factors = [1,2,3,4,5]
decays = [0.1,0.01,0.001]
LR = [1e-6, 1e-5, 1e-4, 1e-3]
fusion = 0.5

In [272]:
checks = np.nonzero(interactions_train_orig)
checks_v = np.nonzero(interactions_test_orig)

In [345]:
#original ratings
best_val_loss = 1000.0
best_train_loss = 1000.0
accur = 0
model = MatrixFactorization(interactions_train_orig.shape[0], interactions_train_orig.shape[1], n_factors=4)
loss_func = torch.nn.MSELoss()
reg_loss_func = torch.optim.Adam(model.parameters(), lr=1e-6, weight_decay=0.001)
for i in range(5):
    print ("Epoch: " + str(i))
    running_loss = 0.0
    accur = 0
    #best_train_loss = 1000.0
    for m, batch in enumerate(get_batch(batch_size, interactions_train_orig)):
        reg_loss_func.zero_grad()
        interactions = Variable(torch.FloatTensor(interactions_train_orig[batch,:].toarray()))
        rows = Variable(torch.LongTensor(batch))
        cols = Variable(torch.LongTensor(np.arange(interactions_train_orig.shape[1])))
        first = (context_train[batch,:].toarray())*fusion
        
        #contexts = Variable(torch.FloatTensor(contexts_full[batch,:].toarray()))
        contexts = Variable(torch.FloatTensor(first))
        
        predictions = model(rows, cols, contexts)
        
        loss = loss_func(predictions, interactions)
        running_loss += np.sqrt(loss.data[0])*batch_size
                
        #correct = get_accuracy_test(interactions_train_orig, model, rows, fusion, contexts_full)
        correct = get_accuracy_test(interactions_train_orig, model, rows, fusion, context_train)
        accur = accur + correct
                
        loss.backward()
    reg_loss_func.step()
    
    epoch_loss = running_loss/interactions_train_orig.shape[0]
    print ('train_loss: ' + str(epoch_loss))
            
    print ('accuracy:' + str(accur/len(checks[0])))
            
    if epoch_loss < best_train_loss:
        best_train_loss = epoch_loss
        print ('checkpoint model with train loss: ' + str(best_train_loss))
        checkpoint_model(epoch_loss, model, save_path)
        
    val_loss, val_accuracy = run_validation(model,interactions_test_orig, context_test,fusion, batch_size, save_path)
    #val_loss, val_accuracy = run_validation(model,interactions_test_orig, contexts_full, fusion, batch_size, save_path)
    print ('val loss: ' + str(val_loss))
    print ('val accuracy: ' + str(val_accuracy/len(checks_v[0])))
        
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        print ('checkpointing_model with val loss: ' + str(best_val_loss))
        checkpoint_model(val_loss, model, save_path_val)

Epoch: 0
train_loss: 2.103221133048775
accuracy:0.38522012578616355
checkpoint model with train loss: 2.103221133048775
val loss: 2.036334830806478
val accuracy: 0.3726937269372694
checkpointing_model with val loss: 2.036334830806478
Epoch: 1
train_loss: 2.0867479995418434
accuracy:0.3915094339622642
checkpoint model with train loss: 2.0867479995418434
val loss: 2.053281370530592
val accuracy: 0.3800738007380074
Epoch: 2
train_loss: 2.0938227490696297
accuracy:0.3836477987421384
val loss: 2.0547341930844865
val accuracy: 0.34317343173431736
Epoch: 3
train_loss: 2.1091754072122986
accuracy:0.39308176100628933
val loss: 2.017407782406495
val accuracy: 0.3284132841328413
checkpointing_model with val loss: 2.017407782406495
Epoch: 4
train_loss: 2.0901668377161173
accuracy:0.35062893081761004
val loss: 2.0372075748276908
val accuracy: 0.34686346863468637


In [342]:
mean = np.mean(interactions_train_orig)
save_path_b = './project_models/bmodel.ckpt'
save_path_val_b = './project_models/bmodelv.ckpt'

In [343]:
fusion = 1.0
#original ratings
best_val_loss = 1000.0
best_train_loss = 1000.0
accur = 0
model = BiasedMatrixFactorization(interactions_train_orig.shape[0], interactions_train_orig.shape[1], mean, n_factors=4)
loss_func = torch.nn.MSELoss()
reg_loss_func = torch.optim.Adam(model.parameters(), lr=1e-6, weight_decay=0.001)
for i in range(5):
    print ("Epoch: " + str(i))
    running_loss = 0.0
    accur = 0
    #best_train_loss = 1000.0
    for m, batch in enumerate(get_batch(batch_size, interactions_train_orig)):
        reg_loss_func.zero_grad()
        interactions = Variable(torch.FloatTensor(interactions_train_orig[batch,:].toarray()))
        rows = Variable(torch.LongTensor(batch))
        cols = Variable(torch.LongTensor(np.arange(interactions_train_orig.shape[1])))
        first = (contexts_full[batch,:].toarray())*fusion
        
        #contexts = Variable(torch.FloatTensor(contexts_full[batch,:].toarray()))
        contexts = Variable(torch.FloatTensor(first))
        
        predictions = model(rows, cols, contexts)
        
        loss = loss_func(predictions, interactions)
        running_loss += np.sqrt(loss.data[0])*batch_size
                
        correct = get_accuracy_test(interactions_train_orig, model, rows, fusion, contexts_full)
        #correct = get_accuracy_test(interactions_train_orig, model, rows, context_train)
        accur = accur + correct
                
        loss.backward()
    reg_loss_func.step()
    
    epoch_loss = running_loss/interactions_train_orig.shape[0]
    print ('train_loss: ' + str(epoch_loss))
            
    print ('accuracy:' + str(accur/len(checks[0])))
            
    if epoch_loss < best_train_loss:
        best_train_loss = epoch_loss
        print ('checkpoint model with train loss: ' + str(best_train_loss))
        checkpoint_model(epoch_loss, model, save_path_b)
        
    #val_loss, val_accuracy = run_validation(model,interactions_test_orig, context_train, batch_size, save_path)
    val_loss, val_accuracy = run_validation(model,interactions_test_orig, contexts_full, fusion, batch_size, save_path_b)
    print ('val loss: ' + str(val_loss))
    print ('val accuracy: ' + str(val_accuracy/len(checks_v[0])))
        
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        print ('checkpointing_model with val loss: ' + str(best_val_loss))
        checkpoint_model(val_loss, model, save_path_val_b)

Epoch: 0
train_loss: 4.550172418345672
accuracy:0.4229559748427673
checkpoint model with train loss: 4.550172418345672
val loss: 4.636757545310604
val accuracy: 0.36531365313653136
checkpointing_model with val loss: 4.636757545310604
Epoch: 1
train_loss: 4.615890558419431
accuracy:0.4371069182389937
val loss: 4.598108047392766
val accuracy: 0.3763837638376384
checkpointing_model with val loss: 4.598108047392766
Epoch: 2
train_loss: 4.635877704357542
accuracy:0.4449685534591195
val loss: 4.63508328878423
val accuracy: 0.35424354243542433
Epoch: 3
train_loss: 4.621652008879292
accuracy:0.4229559748427673
val loss: 4.627915499152715
val accuracy: 0.3837638376383764
Epoch: 4
train_loss: 4.583118679920118
accuracy:0.41509433962264153
val loss: 4.632746507668794
val accuracy: 0.34686346863468637
